In [ ]:
#Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1.Extracting representative frames from video (10 equally spaced frames).

In [ ]:
!pip install opencv-python ffmpeg-python


In [ ]:
#Extract equally spaced video frames

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

def get_video_stats(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video {video_path}")
        return None

    # Get total frames and FPS
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    print(f"Video Path: {video_path}")
    print(f"Total Frames: {total_frames}")
    print(f"FPS: {fps}")

    duration = total_frames / fps if fps > 0 else None
    if duration is None:
        print("FPS could not be retrieved.")

    cap.release()
    return duration

def extract_frames(video_path, num_frames=10):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    frames = []
    idx_set = set(frame_indices)
    current_idx = 0
    grabbed_frames = 0

    while grabbed_frames < num_frames and cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            if current_idx in idx_set:
                frames.append(frame)
                grabbed_frames += 1
            current_idx += 1

    cap.release()
    return frames, frame_indices

def display_frames(frames):
    for i, frame in enumerate(frames):
        plt.figure(figsize=(10, 6))
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.title(f'Frame {i+1}')
        plt.axis('off')
        plt.show()

def save_frames(frames, video_path, frame_indices):
    base_name = os.path.splitext(os.path.basename(video_path))[0] #video basename
    output_dir = os.path.join('/content/drive/MyDrive/summaries/selected/', base_name)
    os.makedirs(output_dir, exist_ok=True)

    for i, frame in enumerate(frames):
        frame_index = frame_indices[i]
        frame_filename = os.path.join(output_dir, f"{base_name}_frame_{frame_index}.png")
        cv2.imwrite(frame_filename, frame)
        print(f"Saved frame {i+1} to {frame_filename}")



In [ ]:
#From a folder of videos, pick random video under 10 minutes and extract frames.

def walk_and_process_videos(root_folder):
    video_files = []

    # Walk through the directory and collect all video files
    for dirpath, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.lower().endswith(('.mp4', '.avi', '.mov', '.mkv', '.flv')):
                video_files.append(os.path.join(dirpath, filename))

    if not video_files:
        print("No video files found.")
        return

    random.shuffle(video_files)

    for video_path in video_files:
        duration = get_video_stats(video_path)
        if duration is not None and duration < 600: #10 minute max
            print(f"Processing {os.path.basename(video_path)} (Duration: {duration/60:.2f} minutes)")
            frames, frame_indices = extract_frames(video_path)
            display_frames(frames)

            # Ask the user if they want to save the frames
            save = input("Do you want to save these frames? (yes/no): ").strip().lower()
            if save == 'yes':
                save_frames(frames, video_path, frame_indices)
            break
    else:
        print("No video under 10 minutes was found.")


In [ ]:
video_path = '/content/drive/MyDrive/summaries/selected/Incidents-00001040.MP4'
frames, frame_indices = extract_frames(video_path)
save_frames(frames, video_path, frame_indices)

[  0  50 101 152 202 253 304 354 405 456]
Saved frame 1 to /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040_frame_0.png
Saved frame 2 to /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040_frame_50.png
Saved frame 3 to /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040_frame_101.png
Saved frame 4 to /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040_frame_152.png
Saved frame 5 to /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040_frame_202.png
Saved frame 6 to /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040_frame_253.png
Saved frame 7 to /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040_frame_304.png
Saved frame 8 to /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040_frame_354.png
Saved frame 9 to /content/drive/MyDrive/summaries/selected/Incidents-0000

In [ ]:
root_folder = '/content/drive/MyDrive/COPY'

walk_and_process_videos(root_folder)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def check_if_video_processed(video_path):
    base_name = os.path.splitext(os.path.basename(video_path))[0]
    output_dir = os.path.join('/content/drive/MyDrive/summaries', base_name)
    return os.path.exists(output_dir)

def walk_and_process_videos(root_folder, sample_size=100):
    video_files = []

    for dirpath, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.lower().endswith(('.mp4', '.avi', '.mov', '.mkv', '.flv')):
                video_files.append(os.path.join(dirpath, filename))

    if not video_files:
        print("No video files found.")
        return

    # Randomly shuffle list of video files
    random.shuffle(video_files)

    selected_videos = []
    for video_path in video_files:
        if len(selected_videos) >= sample_size:
            break
        if check_if_video_processed(video_path):
            continue

        duration = get_video_stats(video_path)
        if duration is not None and duration < 600:
            selected_videos.append(video_path)

    if not selected_videos:
        print("No unprocessed videos under 10 minutes found.")
        return

    for video_path in selected_videos:
        print(f"Processing {os.path.basename(video_path)} (Duration: {get_video_stats(video_path)/60:.2f} minutes)")
        frames = extract_frames(video_path)
        #display_frames(frames)
        save_frames(frames, video_path)



In [ ]:
walk_and_process_videos('/content/drive/MyDrive/COPY', sample_size=100)


NameError: name 'walk_and_process_videos' is not defined

# 2. Passing frames to Gemini 1.5

In [ ]:
!pip install -q -U google-generativeai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 25.3 MB/s eta 0:00:00


In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
GOOGLE_API_KEY='key'#userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
import os
from PIL import Image

def create_ordered_images(frames_dir, limit=10):
    images = []
    for filename in os.listdir(frames_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(frames_dir, filename)
            try:
                img = Image.open(file_path)
                file_number = int(filename.split('_frame_')[1].split('.')[0])
                images.append((file_number, img))
                print(f"Opened image {file_path} as img{file_number}")
            except Exception as e:
                print(f"Error opening image {file_path}: {e}")

    # Sort images by file_number
    images.sort(key=lambda x: x[0])

    ordered_images = [img for _, img in images[:limit]]

    return ordered_images



In [ ]:
frames_dir = '/content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040'
ordered_images = create_ordered_images(frames_dir)


Opened image /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040/Incidents-00001040_frame_0.png as img0
Opened image /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040/Incidents-00001040_frame_50.png as img50
Opened image /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040/Incidents-00001040_frame_101.png as img101
Opened image /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040/Incidents-00001040_frame_152.png as img152
Opened image /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040/Incidents-00001040_frame_202.png as img202
Opened image /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040/Incidents-00001040_frame_253.png as img253
Opened image /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040/Incidents-00001040_frame_304.png as img304
Opened image /content/drive/MyDrive/summaries/selected

In [ ]:
#test

model = genai.GenerativeModel('gemini-1.5-flash')

response = model.generate_content(["Describe this sequence of images of the NYPD at a protest and pay attention to the actions of people, any details on the location, identifying details like uniforms and badges, and any signs, numbers, and text in the images."] + ordered_images)
to_markdown(response.text)


> The images show a line of NYPD officers in riot gear facing a crowd of protesters. The officers are wearing black uniforms, helmets with visors, and face masks. They are carrying batons. The protesters are wearing a variety of clothing and some are also wearing face masks. One officer is wearing a badge that reads "POLICE" and has the number 17945 visible. The location appears to be a street in New York City, with streetlights and buildings visible in the background. The street signs visible say "Bedford Ave". There are several people holding up their phones and recording the confrontation. One sign in the background of the first image says "Black Lives Matter".  The officers are moving towards the crowd, while some protesters appear to be backing away. Some protesters are holding their hands up in the air. This sequence of images shows a tense encounter between the NYPD and protesters. 


In [ ]:
def save_string_to_file(text, folder_path, file_name):
    os.makedirs(folder_path, exist_ok=True)
    file_path = os.path.join(folder_path, file_name)

    with open(file_path, 'w') as file:
        file.write(text)

    print(f"File saved at: {file_path}")
    return file_path


folder_path = frames_dir #'/content/drive/MyDrive/summaries/selected/DEF_000321027/DEF_000321027_1987-2076'
file_name = 'gemini_1_5_flash.txt'
text_content = response.text

save_string_to_file(text_content, folder_path, file_name)


File saved at: /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040/gemini_1_5_flash.txt


'/content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040/gemini_1_5_flash.txt'

In [ ]:
import os
import json

def process_directory(base_dir, log_file='processed_folders.json', model_name='gemini-1.5-flash', output_file_name='gemini_1_5_flash.txt'):
    # Load the list of processed folders from the log file
    processed_folders = load_processed_folders(log_file)
    skip_folders = {'done', 'selected'}

    for folder_name in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder_name)

        if os.path.isdir(folder_path) and folder_name not in skip_folders:
            if folder_name not in processed_folders:
                output_file_path = os.path.join(folder_path, output_file_name)
                if not os.path.exists(output_file_path):
                    try:
                        process_images_and_generate_summary(folder_path, model_name, output_file_name)
                        processed_folders.append(folder_name)
                        save_processed_folders(log_file, processed_folders)
                        print(f"Processed and saved summary for folder: {folder_name}")
                    except Exception as e:
                        print(f"Error processing folder {folder_name}: {e}")
                else:
                    print(f"Summary already exists for folder: {folder_name}")
            else:
                print(f"Folder {folder_name} already processed.")

def process_images_and_generate_summary(frames_dir, model_name='gemini-1.5-flash', output_file_name='gemini_1_5_flash.txt'):
    os.makedirs(frames_dir, exist_ok=True)
    ordered_images = create_ordered_images(frames_dir)
    model = genai.GenerativeModel(model_name)
    response = model.generate_content(
        ["Describe this sequence of images of from a video of the NYPD at a protest and pay attention to the actions of people, any details on the location, identifying details like uniforms and badges, and any signs, numbers, and text in the images."] + ordered_images
    )
    return save_string_to_file(response.text, frames_dir, output_file_name)

def load_processed_folders(log_file):

    if os.path.exists(log_file):
        with open(log_file, 'r') as file:
            return json.load(file)
    return []

def save_processed_folders(log_file, processed_folders):

    with open(log_file, 'w') as file:
        json.dump(processed_folders, file)




In [ ]:
#Run on extracted frames dir
base_dir = '/content/drive/MyDrive/summaries'
process_directory(base_dir)

Opened image /content/drive/MyDrive/summaries/Incidents-00000194/Incidents-00000194_frame_1.png as img1
Opened image /content/drive/MyDrive/summaries/Incidents-00000194/Incidents-00000194_frame_2.png as img2
Opened image /content/drive/MyDrive/summaries/Incidents-00000194/Incidents-00000194_frame_3.png as img3
Opened image /content/drive/MyDrive/summaries/Incidents-00000194/Incidents-00000194_frame_4.png as img4
Opened image /content/drive/MyDrive/summaries/Incidents-00000194/Incidents-00000194_frame_5.png as img5
Opened image /content/drive/MyDrive/summaries/Incidents-00000194/Incidents-00000194_frame_6.png as img6
Opened image /content/drive/MyDrive/summaries/Incidents-00000194/Incidents-00000194_frame_7.png as img7
Opened image /content/drive/MyDrive/summaries/Incidents-00000194/Incidents-00000194_frame_8.png as img8
Opened image /content/drive/MyDrive/summaries/Incidents-00000194/Incidents-00000194_frame_9.png as img9
Opened image /content/drive/MyDrive/summaries/Incidents-00000194

Error processing folder Incidents-00000194: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Request payload size exceeds the limit: 20971520 bytes.
Summary already exists for folder: NYPD_N_00001085
Summary already exists for folder: MANDA_00000034
Opened image /content/drive/MyDrive/summaries/Incidents-00001862/Incidents-00001862_frame_1.png as img1
Opened image /content/drive/MyDrive/summaries/Incidents-00001862/Incidents-00001862_frame_2.png as img2
Opened image /content/drive/MyDrive/summaries/Incidents-00001862/Incidents-00001862_frame_3.png as img3
Opened image /content/drive/MyDrive/summaries/Incidents-00001862/Incidents-00001862_frame_4.png as img4
Opened image /content/drive/MyDrive/summaries/Incidents-00001862/Incidents-00001862_frame_5.png as img5
Opened image /content/drive/MyDrive/summaries/Incidents-00001862/Incidents-00001862_frame_6.png as img6
Opened image /content/drive/MyDrive/summar

Error processing folder Incidents-00001862: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Request payload size exceeds the limit: 20971520 bytes.
Opened image /content/drive/MyDrive/summaries/Incidents-00002043/Incidents-00002043_frame_1.png as img1
Opened image /content/drive/MyDrive/summaries/Incidents-00002043/Incidents-00002043_frame_2.png as img2
Opened image /content/drive/MyDrive/summaries/Incidents-00002043/Incidents-00002043_frame_3.png as img3
Opened image /content/drive/MyDrive/summaries/Incidents-00002043/Incidents-00002043_frame_4.png as img4
Opened image /content/drive/MyDrive/summaries/Incidents-00002043/Incidents-00002043_frame_5.png as img5
Opened image /content/drive/MyDrive/summaries/Incidents-00002043/Incidents-00002043_frame_6.png as img6
Opened image /content/drive/MyDrive/summaries/Incidents-00002043/Incidents-00002043_frame_7.png as img7
Opened image /content/drive/MyDrive/sum

Error processing folder Incidents-00002043: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Request payload size exceeds the limit: 20971520 bytes.
Summary already exists for folder: DEF_000320313
Summary already exists for folder: DEF_000514869
Summary already exists for folder: DEF_000322502
Summary already exists for folder: MANDA_00000143
Summary already exists for folder: DEF_000320178
Summary already exists for folder: DEF_000276544
Summary already exists for folder: MANDA_00000258
Summary already exists for folder: NYPD_N_00000619
Summary already exists for folder: DEF_000320733
Summary already exists for folder: NYPD_N_00000363
Summary already exists for folder: DEF_000322222
Summary already exists for folder: NYPD-0000096560
Summary already exists for folder: Incidents-00000891
Opened image /content/drive/MyDrive/summaries/Incidents-00002091/Incidents-00002091_frame_1.png as img1
Opened image 

Error processing folder Incidents-00002091: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Request payload size exceeds the limit: 20971520 bytes.
Summary already exists for folder: MANDA_00000079
Summary already exists for folder: DEF_000320628
Opened image /content/drive/MyDrive/summaries/Hearing-00000059/Hearing-00000059_frame_1.png as img1
Opened image /content/drive/MyDrive/summaries/Hearing-00000059/Hearing-00000059_frame_2.png as img2
Opened image /content/drive/MyDrive/summaries/Hearing-00000059/Hearing-00000059_frame_3.png as img3
Opened image /content/drive/MyDrive/summaries/Hearing-00000059/Hearing-00000059_frame_4.png as img4
Opened image /content/drive/MyDrive/summaries/Hearing-00000059/Hearing-00000059_frame_5.png as img5
Opened image /content/drive/MyDrive/summaries/Hearing-00000059/Hearing-00000059_frame_6.png as img6
Opened image /content/drive/MyDrive/summaries/Hearing-00000059/Heari

...

Error processing folder Incidents-00002396: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Request payload size exceeds the limit: 20971520 bytes.
Summary already exists for folder: DEF_000277036
Summary already exists for folder: NYPD_N_00001263
Summary already exists for folder: DEF_000517426
Opened image /content/drive/MyDrive/summaries/DEFVID_000000149/DEFVID_000000149_frame_1.png as img1
Opened image /content/drive/MyDrive/summaries/DEFVID_000000149/DEFVID_000000149_frame_2.png as img2
Opened image /content/drive/MyDrive/summaries/DEFVID_000000149/DEFVID_000000149_frame_3.png as img3
Opened image /content/drive/MyDrive/summaries/DEFVID_000000149/DEFVID_000000149_frame_4.png as img4
Opened image /content/drive/MyDrive/summaries/DEFVID_000000149/DEFVID_000000149_frame_5.png as img5
Opened image /content/drive/MyDrive/summaries/DEFVID_000000149/DEFVID_000000149_frame_6.png as img6
Opened image /cont

Error processing folder DEFVID_000000149: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Request payload size exceeds the limit: 20971520 bytes.
Summary already exists for folder: DEF_000269465
Summary already exists for folder: DEF_000276714
Summary already exists for folder: DEF_000322574
Summary already exists for folder: DEF_000321295
Summary already exists for folder: Incidents-00001935
Summary already exists for folder: Incidents-00001691
Summary already exists for folder: DEF_000518651
Summary already exists for folder: DEF_000321278
Summary already exists for folder: DEF_000514911
Summary already exists for folder: NYPD-0000104490
Summary already exists for folder: DEF_000269568
Summary already exists for folder: DEF_000461448
Summary already exists for folder: Gut 3.mp4
Summary already exists for folder: DEF_000277078
Opened image /content/drive/MyDrive/summaries/KHernandez-00000126/KHernande

Error processing folder KHernandez-00000126: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Request payload size exceeds the limit: 20971520 bytes.
Summary already exists for folder: DEF_000277119
Summary already exists for folder: DEF_E_PD_00035902
Summary already exists for folder: DEF_000320695
Summary already exists for folder: NYPD_N_00001167
Summary already exists for folder: DEF_000514802
Summary already exists for folder: NYPD-0000113508
Summary already exists for folder: Incidents-00000583
Summary already exists for folder: NYPD_N_00000655
Opened image /content/drive/MyDrive/summaries/Incidents-00002038/Incidents-00002038_frame_1.png as img1
Opened image /content/drive/MyDrive/summaries/Incidents-00002038/Incidents-00002038_frame_2.png as img2
Opened image /content/drive/MyDrive/summaries/Incidents-00002038/Incidents-00002038_frame_3.png as img3
Opened image /content/drive/MyDrive/summaries/I

Error processing folder Incidents-00002038: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Request payload size exceeds the limit: 20971520 bytes.
Opened image /content/drive/MyDrive/summaries/Incidents-00002014/Incidents-00002014_frame_1.png as img1
Opened image /content/drive/MyDrive/summaries/Incidents-00002014/Incidents-00002014_frame_2.png as img2
Opened image /content/drive/MyDrive/summaries/Incidents-00002014/Incidents-00002014_frame_3.png as img3
Opened image /content/drive/MyDrive/summaries/Incidents-00002014/Incidents-00002014_frame_4.png as img4
Opened image /content/drive/MyDrive/summaries/Incidents-00002014/Incidents-00002014_frame_5.png as img5
Opened image /content/drive/MyDrive/summaries/Incidents-00002014/Incidents-00002014_frame_6.png as img6
Opened image /content/drive/MyDrive/summaries/Incidents-00002014/Incidents-00002014_frame_7.png as img7
Opened image /content/drive/MyDrive/sum

Error processing folder Incidents-00002014: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Request payload size exceeds the limit: 20971520 bytes.
Opened image /content/drive/MyDrive/summaries/Incidents-00001881/Incidents-00001881_frame_1.png as img1
Opened image /content/drive/MyDrive/summaries/Incidents-00001881/Incidents-00001881_frame_2.png as img2
Opened image /content/drive/MyDrive/summaries/Incidents-00001881/Incidents-00001881_frame_3.png as img3
Opened image /content/drive/MyDrive/summaries/Incidents-00001881/Incidents-00001881_frame_4.png as img4
Opened image /content/drive/MyDrive/summaries/Incidents-00001881/Incidents-00001881_frame_5.png as img5
Opened image /content/drive/MyDrive/summaries/Incidents-00001881/Incidents-00001881_frame_6.png as img6
Opened image /content/drive/MyDrive/summaries/Incidents-00001881/Incidents-00001881_frame_7.png as img7
Opened image /content/drive/MyDrive/sum

# 3. Passing frames to ChatGPT

In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
import base64
import requests

api_key = "key"

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
import os
image_folder = frames_dir#"/content/drive/MyDrive/summaries/selected/DEF_000321027/DEF_000321027_1987-2076"
limit = 10
sequence = []

for filename in os.listdir(image_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        file_path = os.path.join(image_folder, filename)
        file_number = int(filename.split('_frame_')[1].split('.')[0])
        sequence.append((file_number, file_path))
        print(f"Added image {file_path} as img{file_number}")
sequence.sort(key=lambda x: x[0])
ordered_images = [file_path for _, file_path in sequence[:limit]]

image_contents = []

for image_path in ordered_images:
    base64_image = encode_image(image_path)
    image_contents.append(
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
            }
        }
    )

Added image /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040_frame_0.png as img0
Added image /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040_frame_50.png as img50
Added image /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040_frame_101.png as img101
Added image /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040_frame_152.png as img152
Added image /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040_frame_202.png as img202
Added image /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040_frame_253.png as img253
Added image /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040_frame_304.png as img304
Added image /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incidents-00001040_frame_354.png as img354
Added image /content/drive/MyDrive/summaries/selected/Incidents-00001040/Incid

In [ ]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

payload = {
    "model": "gpt-4o-mini",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Describe this sequence of images of the NYPD at a protest and pay attention to the actions of people, any details on the location, identifying details like uniforms and badges, and any signs, numbers, and text in the images."
                }
            ] + image_contents
        }
    ],
    "max_tokens": 600
}

print(payload)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#test
response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
text = response.json()['choices'][0]['message']['content']
print(text)

The sequence of images depicts a significant protest scene involving the NYPD. Here’s a detailed description based on the observed elements:

1. **NYPD Presence**: Officers are prominently positioned in riot gear, wearing helmets with visors and uniforms indicating their affiliation with the NYPD. Notably, some officers have numbers displayed on their helmets (e.g., 17945, 22244).

2. **Protesters**: Several individuals are visible in the foreground, many wearing masks. They exhibit expressions ranging from distress to agitation, suggesting a tense atmosphere. Some appear to be shielding their faces, possibly due to the presence of irritants like tear gas or other crowd control measures.

3. **Actions and Reactions**: 
   - In the early images, an officer is seen gesturing, possibly directing or communicating with the crowd. This hand signal may indicate an order or warning.
   - In subsequent images, groups of protesters appear to be fleeing or moving away from the police line, which 

In [ ]:
import os

def save_string_to_file(text, folder_path, file_name):
    os.makedirs(folder_path, exist_ok=True)
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'w') as file:
        file.write(text)

    print(f"File saved at: {file_path}")
    return file_path

folder_path = frames_dir #'/content/drive/MyDrive/summaries/selected/DEF_000321027/DEF_000321027_1987-2076'
file_name = 'gpt_4o_mini.txt'
text_content = text

save_string_to_file(text_content, folder_path, file_name)


File saved at: /content/drive/MyDrive/summaries/selected/DEF_000321027/DEF_000321027_8250-8685/gpt_4o_mini.txt


'/content/drive/MyDrive/summaries/selected/DEF_000321027/DEF_000321027_8250-8685/gpt_4o_mini.txt'

In [ ]:
import os
import requests

def process_directory(base_dir, limit=10, model_name='gpt-4o-mini', output_file_name='gpt_4o_mini.txt', api_key=None):

    skip_folders = {'done', 'selected'}
    for folder_name in os.listdir(base_dir) and folder_name not in skip_folders:
        folder_path = os.path.join(base_dir, folder_name)
        if os.path.isdir(folder_path):
            output_file_path = os.path.join(folder_path, output_file_name)
            if not os.path.exists(output_file_path):
                try:
                    sequence = []

                    for filename in os.listdir(folder_path):
                        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                            file_path = os.path.join(folder_path, filename)
                            file_number = int(filename.split('_frame_')[1].split('.')[0])
                            sequence.append((file_number, file_path))
                            print(f"Added image {file_path} as img{file_number}")

                    sequence.sort(key=lambda x: x[0])
                    ordered_images = [file_path for _, file_path in sequence[:limit]]

                    image_contents = []

                    for image_path in ordered_images:
                        base64_image = encode_image(image_path)
                        image_contents.append(
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/jpeg;base64,{base64_image}"
                                }
                            }
                        )

                    headers = {
                        "Content-Type": "application/json",
                        "Authorization": f"Bearer {api_key}"
                    }

                    payload = {
                        "model": model_name,
                        "messages": [
                            {
                                "role": "user",
                                "content": [
                                    {
                                        "type": "text",
                                        "text": "Describe this sequence of images of the NYPD at a protest and pay attention to the actions of people, any details on the location, identifying details like uniforms and badges, and any signs, numbers, and text in the images."
                                    }
                                ] + image_contents
                            }
                        ],
                        "max_tokens": 600
                    }

                    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
                    text = response.json()['choices'][0]['message']['content']
                    print(text)

                    save_string_to_file(text, folder_path, output_file_name)

                except Exception as e:
                    print(f"Error processing folder {folder_name}: {e}")
            else:
                print(f"Summary already exists for folder: {folder_name}")

def save_string_to_file(text, folder_path, file_name):
    os.makedirs(folder_path, exist_ok=True)
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'w') as file:
        file.write(text)

    print(f"File saved at: {file_path}")
    return file_path




In [ ]:
#Run on extracted frames dir
base_dir = '/content/drive/MyDrive/summaries'
api_key = "key"

process_directory(base_dir, api_key=api_key)